In [1]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score
from joblib import dump, load

%load_ext autoreload
%autoreload 2

In [2]:
# %%capture
# convert_data()


In [3]:
i = 7

x_train, y_train = read_file("idle_train", f"A0{i}")
x_test, y_test = read_file("idle_test", f"A0{i}")


print("final shapes are: ", x_train.shape, x_test.shape, y_train.shape, y_test.shape)


final shapes are:  (336, 25, 750) (85, 25, 750) (336,) (85,)


In [4]:
%%capture
train_coeff = featurize(x_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
x_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)



In [5]:
test_coeff = featurize(x_test)
x_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)


In [6]:
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(x_train_f, y_train)

print("Accuracy is ", clf.score(x_train_f, y_train))
print(len(x_train))

print("test Accuracy is ", clf.score(x_test_f ,y_test))
print(len(x_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(x_train_f), y_train))
print("kappa score on test is: ", cohen_kappa_score(clf.predict(x_test_f), y_test))


Accuracy is  1.0
336
test Accuracy is  1.0
85
kappa score on train is:  1.0
kappa score on test is:  1.0


In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(classification_report(y_test, clf.predict(x_test_f)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        27
         1.0       1.00      1.00      1.00        58

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



In [8]:
dump(clf, "idle_model.joblib")
dump(csp, "idle_csp.joblib")

['idle_csp.joblib']